In [1]:
###Chris
import pandas as pd
import numpy as np
import glob
import os
import re

#Matt
# Run in python console
import nltk; nltk.download('stopwords')

import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/paperspace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df_jobs = pd.read_csv('./data/jobs_df.csv')
duties = df_jobs.job_duties
data = duties

In [3]:
# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

In [4]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

In [5]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [6]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [7]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [8]:
#Add Stop Word Extensions
stop_words.extend(['may', 'los', 'angeles', 'work', 'use', 'equal', 'opportunity'])

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [9]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [10]:
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
 
NUM_TOPICS = 10

vectorizer = CountVectorizer(min_df=5, max_df=0.9, 
                             stop_words='english', lowercase=True, 
                             token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(duties)

In [11]:
nmf_model = NMF(n_components=NUM_TOPICS)
nmf_1 = nmf_model.fit(data_vectorized)
nmf_t = nmf_model.fit_transform(data_vectorized)

In [12]:
def print_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])
print("NMF Model:")
print_topics(nmf_1, vectorizer)
print("=" * 20)

NMF Model:
Topic 0:
[('city', 2.7824310768154703), ('angeles', 2.6864405701066936), ('los', 2.6864405701066936), ('candidates', 2.3794747091361557), ('test', 2.096804035851401), ('experience', 1.8612573160465795), ('lafd', 1.790568286234538), ('disability', 1.7016082514233062), ('department', 1.4672642911058227), ('chief', 1.4629534128538648)]
Topic 1:
[('work', 2.492905370018081), ('building', 2.097925570914508), ('maintaining', 1.9885348482345504), ('applies', 1.9087664531245698), ('techniques', 1.8896438024103595), ('effective', 1.8877565736619901), ('supervisory', 1.8705230420538637), ('principles', 1.8669980790196719), ('sound', 1.8647007645841318), ('responsibilities', 1.8579652011567849)]
Topic 2:
[('equipment', 3.6823798975531106), ('electrical', 2.4529641149799843), ('maintenance', 1.8585171279184658), ('electric', 1.4665245145682912), ('repair', 1.3877200921251238), ('construction', 1.2310447134688403), ('distribution', 1.1574699010969822), ('lines', 0.9528765832422358), ('po

array([[0.        , 0.69946   , 0.37114811, ..., 0.91558672, 0.        ,
        0.00464671],
       [0.        , 0.        , 0.        , ..., 0.        , 0.06565122,
        0.01982516],
       [0.03446464, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.02007377, 0.        , 0.        , ..., 0.        , 0.04367815,
        0.        ],
       [0.        , 0.24126238, 0.10032013, ..., 0.25762635, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.27556406]])

In [14]:
coherence_model_ldamallet = CoherenceModel(model=nmf_1.components_, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

ValueError: This topic model is not currently supported. Supported topic models should implement the `get_topics` method.

In [15]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [16]:
from gensim.test.utils import common_corpus, common_dictionary
from gensim.models.wrappers import LdaMallet
mallet_path = '~/data/mallet-2.0.8/bin/mallet'

start = 2
limit = 40
step = 6
model_list, coherence_values = compute_coherence_values(dictionary=nmf_t, corpus=corpus, texts=duties, start=start, limit=limit, step=step)

/home/paperspace/.local/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


AttributeError: 'numpy.ndarray' object has no attribute 'id2token'

In [17]:
from gensim import models
from gensim.corpora.dictionary import Dictionary

common_dictionary = Dictionary(data_lemmatized)
common_corpus = [common_dictionary.doc2bow(text) for text in data_lemmatized]

In [24]:
nmf_1 = gensim.models.nmf.Nmf(common_corpus, num_topics=10, normalize=True)

AttributeError: module 'gensim.models' has no attribute 'nmf'

In [ ]:
cm = CoherenceModel(model=nfm_1, corpus=common_corpus, coherence='u_mass')

In [ ]:
train_headlines_sentences = [' '.join(text) for text in train_headlines]


# Now, we obtain a Counts design matrix, for which we use SKLearn’s CountVectorizer module. The transformation will return a matrix of size (Documents x Features), where the value of a cell is going to be the number of times the feature (word) appears in that document.
# 
# To reduce the size of the matrix, to speed up computation, we will set the maximum feature size to 5000, which will take the top 5000 best features that can contribute to our model.

# In[80]:

vectorizer = CountVectorizer(analyzer='word', max_features=5000);
x_counts = vectorizer.fit_transform(train_headlines_sentences);


# Next, we set a TfIdf Transformer, and transform the counts with the model.

# In[81]:

transformer = TfidfTransformer(smooth_idf=False);
x_tfidf = transformer.fit_transform(x_counts);


# And now we normalize the TfIdf values to unit length for each row.

# In[82]:

xtfidf_norm = normalize(x_tfidf, norm='l1', axis=1)


# And finally, obtain a NMF model, and fit it with the sentences.

# In[84]:

#obtain a NMF model.
model = NMF(n_components=num_topics, init='nndsvd');


# In[85]:

#fit the model
model.fit(xtfidf_norm)


# In[136]:

def get_nmf_topics(model, n_top_words):
    
    #the word ids obtained need to be reverse-mapped to the words so we can print the topic names.
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(num_topics):
        
        #for each topic, obtain the largest values, and add the words they map to into the dictionary.
        words_ids = model.components_[i].argsort()[:-20 - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict);


# In[139]:

get_nmf_topics(model, 20)